In [1]:
!pip install opendatasets


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
# from torchvision.datasets import Imageforlder
import torchvision.datasets as datasets
from torch.utils.data import  random_split, DataLoader
import random
from skimage import io
import matplotlib.pyplot as plt
# para opendataset
import opendatasets as od
from fastai.vision.all import *
import numpy as np
import torch
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:
# Descargamos el dataset
dataset_link = "https://www.kaggle.com/datasets/nightfury007/fercustomdataset-3classes"
od.download(dataset_link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: carlososmar
Your Kaggle Key: ··········


100%|██████████| 83.1M/83.1M [00:05<00:00, 16.7MB/s]


In [4]:

dataset_dir = "/content/fercustomdataset-3classes/FER_Custom_Dataset"

# Transformaciones al dataset
transform = transforms.Compose([
    # reafirmamos las dimenciones
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])


# Dataset procesado de imagenes almacenadas en folders que lleva por nombre su respectivo label

dataset = datasets.ImageFolder(root= dataset_dir, transform = transform)


# número baches
batch_size= 64


dataloader_ = DataLoader(dataset, batch_size, shuffle= True, pin_memory = True)
classes = dataset.classes
print(classes)



# Calcula el tamaño del conjunto de prueba en función de la proporción
test_size = int(0.2 * len(dataloader_.dataset))

# Calcula el tamaño del conjunto de entrenamiento
train_size = len(dataloader_.dataset) - test_size

# Divide el conjunto de datos en entrenamiento y prueba
train_dataset, test_dataset = random_split(dataloader_.dataset, [train_size, test_size])

# Crea DataLoaders para Test y Train
batch_size = 64
dataloader = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
}





['Disappointed', 'interested', 'neutral']


In [5]:
class_to_idx = dataset.class_to_idx
print(class_to_idx)

{'Disappointed': 0, 'interested': 1, 'neutral': 2}


In [6]:
regNet = models.regnet_x_400mf();
regNet

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2, bias=False)
            

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader




class ModelCustom(torch.nn.Module):
  def __init__(self, n_outputs=5, pretrained=False, freeze=False):
    super().__init__()
    # descargamos regnet
    regNet = models.regnet_x_400mf(weights = pretrained);
    # nos quedamos con todas las capas menos la última
    self.regNet = torch.nn.Sequential(*list(regNet.children())[:-1])
    if freeze:
      for param in self.regNet.parameters():
        param.requires_grad=False
    # añadimos una nueva capa lineal para llevar a cabo la clasificación
    self.fc = torch.nn.Linear(400, 3)

  def forward(self, x):
    x = self.regNet(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

  def unfreeze(self):
    for param in self.regNet.parameters():
        param.requires_grad=True



In [8]:
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5, lr=1e-2):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [22]:
model_c = ModelCustom()
# model_c =  models.regnet_x_400mf(weights = None);
fit(model_c, dataloader)

val_loss 1.13061 val_acc 0.34022: 100%|██████████| 137/137 [00:11<00:00, 11.74it/s]


Epoch 1/5 loss 1.19738 val_loss 1.13061 acc 0.37277 val_acc 0.34022


val_loss 1.12485 val_acc 0.29601: 100%|██████████| 137/137 [00:11<00:00, 11.77it/s]


Epoch 2/5 loss 1.10063 val_loss 1.12485 acc 0.37931 val_acc 0.29601


val_loss 1.10567 val_acc 0.33471: 100%|██████████| 137/137 [00:11<00:00, 12.06it/s]


Epoch 3/5 loss 1.09779 val_loss 1.10567 acc 0.38507 val_acc 0.33471


val_loss 1.10055 val_acc 0.38957: 100%|██████████| 137/137 [00:11<00:00, 11.78it/s]


Epoch 4/5 loss 1.09730 val_loss 1.10055 acc 0.38204 val_acc 0.38957


val_loss 1.09217 val_acc 0.40354: 100%|██████████| 137/137 [00:11<00:00, 11.85it/s]

Epoch 5/5 loss 1.09686 val_loss 1.09217 acc 0.38293 val_acc 0.40354


In [9]:
# Transfer Learnign
model_c = ModelCustom(pretrained=True, freeze=True)
fit(model_c, dataloader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_X_400MF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_X_400MF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/regnet_x_400mf-adf1edd5.pth" to /root/.cache/torch/hub/checkpoints/regnet_x_400mf-adf1edd5.pth
100%|██████████| 21.3M/21.3M [00:00<00:00, 105MB/s] 
val_loss 1.02636 val_acc 0.47849: 100%|██████████| 137/137 [00:12<00:00, 11.35it/s]


Epoch 1/5 loss 1.04337 val_loss 1.02636 acc 0.46943 val_acc 0.47849


val_loss 0.98986 val_acc 0.51085: 100%|██████████| 137/137 [00:11<00:00, 11.58it/s]


Epoch 2/5 loss 1.00565 val_loss 0.98986 acc 0.49924 val_acc 0.51085


val_loss 0.99539 val_acc 0.50128: 100%|██████████| 137/137 [00:11<00:00, 11.52it/s]


Epoch 3/5 loss 0.99838 val_loss 0.99539 acc 0.50261 val_acc 0.50128


val_loss 1.05405 val_acc 0.47374: 100%|██████████| 137/137 [00:11<00:00, 11.69it/s]


Epoch 4/5 loss 0.99626 val_loss 1.05405 acc 0.50646 val_acc 0.47374


val_loss 0.99676 val_acc 0.51261: 100%|██████████| 137/137 [00:11<00:00, 11.69it/s]

Epoch 5/5 loss 0.99256 val_loss 0.99676 acc 0.50917 val_acc 0.51261


In [12]:
# fine tuning
model_c = ModelCustom(pretrained=True, freeze=False)
fit(model_c, dataloader)

val_loss 0.85855 val_acc 0.59361: 100%|██████████| 137/137 [00:12<00:00, 10.84it/s]


Epoch 1/5 loss 0.96068 val_loss 0.85855 acc 0.52721 val_acc 0.59361


loss 0.79628 acc 0.62540:  14%|█▍        | 79/548 [00:09<00:57,  8.13it/s]


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(model, dataloader, device):
    # model.to(device)  # Mueve el modelo al dispositivo correcto
    model.eval()  # Asegúrate de que el modelo esté en modo de evaluación
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch
            X, y = X.to(device), y.to(device)  # Mueve los datos al dispositivo
            y_hat = model(X)

            for i in range(len(y)):
                sample_image = X[i].cpu().numpy()  # Convierte la imagen a numpy (asegúrate de moverla a la CPU si es necesario)
                sample_label = y[i].item()  # Obtiene la etiqueta original como un valor escalar
                predicted_label = torch.argmax(y_hat[i]).item()  # Obtiene la etiqueta predicha
                plt.imshow(np.transpose(sample_image, (1, 2, 0)))  # Cambia las dimensiones para ser (altura, ancho, canales)
                plt.title(f"Original Label: {sample_label}, Predicted Label: {predicted_label}")
                plt.show()


# Llama a la función visualize_predictions después de entrenar tu modelo
# regNet = models.regnet_x_400mf(weights=None)
# fit(regNet, dataloader)
visualize_predictions(model_c, dataloader['test'],device)
